In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
#import statsmodels.tsa.stattools as ts
from init_objects import *
from functions.stylizedfacts import *
from functions.helpers import organise_data
#from functions.evolutionaryalgo import quadratic_loss_function
from functions.inequality import gini, lorenz_curve
from distribution_model import *
import statsmodels.api as sm
from SALib.sample import latin
from matplotlib import style
from collections import OrderedDict
from functions.indirect_calibration import distr_model_performance
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
from functions.find_bubbles import *
from statsmodels.iolib.summary2 import summary_col

## Reasons that boom-busts lead to more inequality

In [2]:
with open('parameters.json', 'r') as f:
    params = json.loads(f.read())

### Reason 1: large redistribution compared to non-boom-bust


1. calc redistr in boom-busts 
2. calc redistri in non-boom bust
3. Compare

##### Calc redistr in boom-bust

In [24]:
NRUNS = 20

In [25]:
params['strat_share_chartists'] = 0.85

In [26]:
trdrs = []
orbs = []
for seed in range(NRUNS): 
    traders, orderbook = init_objects_distr(params, seed)
    traders, orderbook = pb_distr_model(traders, orderbook, params, seed)
    trdrs.append(traders)
    orbs.append(orderbook)

Start of simulation  0
Start of simulation  1
Start of simulation  2
Start of simulation  3
Start of simulation  4
Start of simulation  5
Start of simulation  6
Start of simulation  7
Start of simulation  8
Start of simulation  9
Start of simulation  10
Start of simulation  11
Start of simulation  12
Start of simulation  13
Start of simulation  14
Start of simulation  15
Start of simulation  16
Start of simulation  17
Start of simulation  18
Start of simulation  19


In [27]:
mc_prices, mc_returns, mc_autocorr_returns, mc_autocorr_abs_returns, mc_volatility, mc_volume, mc_fundamentals = organise_data(orbs, burn_in_period=200)

In [28]:
# calc redistribution, profits ... abs 
wealth_starts = []
wealth_ends = []
ginis_ot = []
palmas_ot = []
twtws_ot = []
risk_aversions = []
horizons = []
learning_abilities = []
chartist_expectations = []
fundamentalist_expectations = []
profits = []
real_profits = []
market_returns = []

# determine the start and end wealth of the bubble
for seed, traders in enumerate(trdrs):
    money_start = np.array([x.var.money[0] for x in traders])
    stocks_start = np.array([x.var.stocks[0] for x in traders])
    wealth_start = money_start + (stocks_start * orbs[seed].tick_close_price[0])

    money_end = np.array([x.var.money[-1] for x in traders])
    stocks_end = np.array([x.var.stocks[-1] for x in traders])
    wealth_end = money_end + (stocks_end * orbs[seed].tick_close_price[-1])
    
    # profits
    profits.append((np.array(wealth_end) - np.array(wealth_start)) / np.array(wealth_start))
    real_profits.append(np.array(wealth_end) - np.array(wealth_start))
    market_returns.append([(orbs[seed].tick_close_price[-1] - orbs[seed].tick_close_price[0]) / orbs[seed].tick_close_price[0] for x in range(len(wealth_start))])

    # determine characteristics of the agents
    risk_aversions.append([x.par.risk_aversion for x in traders])
    horizons.append([x.par.horizon for x in traders])
    learning_abilities.append([x.par.learning_ability for x in traders])
    chartist_expectations.append([x.var.weight_chartist[: ] for x in traders])
    fundamentalist_expectations.append([x.var.weight_fundamentalist[:] for x in traders])

    wealth_gini_over_time = []
    palma_over_time = []
    twentytwenty_over_time = []
    for t in range(params['ticks']):
        money = np.array([x.var.money[t] for x in traders])
        stocks = np.array([x.var.stocks[t] for x in traders])
        wealth = money + (stocks * orbs[seed].tick_close_price[0])

        share_top_10 = sum(np.sort(wealth)[int(len(wealth) * 0.9):]) / sum(wealth)
        share_bottom_40 = sum(np.sort(wealth)[:int(len(wealth) * 0.4)]) / sum(wealth)
        palma_over_time.append(share_top_10 / share_bottom_40)

        share_top_20 = np.mean(np.sort(wealth)[int(len(wealth) * 0.8):])
        share_bottom_20 = np.mean(np.sort(wealth)[:int(len(wealth) * 0.2)])
        twentytwenty_over_time.append(share_top_20 / share_bottom_20)

        wealth_gini_over_time.append(gini(wealth))

    wealth_starts.append(list(wealth_start))
    wealth_ends.append(list(wealth_end))
    ginis_ot.append(wealth_gini_over_time)
    palmas_ot.append(palma_over_time)
    twtws_ot.append(twentytwenty_over_time)

In [29]:
redistribution = []
for p in profits:
    redistribution.append(sum(abs(p)))

In [30]:
redistribution

[723.3777348569948,
 900.625620043923,
 944.4615918169923,
 738.0062601898934,
 626.1269302874903,
 711.8962714002488,
 876.6922347666535,
 662.5391429362903,
 693.6416152787484,
 1238.774852857653,
 679.5279880107627,
 532.6346133478758,
 624.5633202275417,
 503.2301963543622,
 519.7031874440195,
 1062.3567380196369,
 655.8470239868523,
 857.3469683619163,
 581.0990512380159,
 970.5717791504254]

##### Calc redistr in non- boom-bust

In [32]:
NRUNS = 20

In [33]:
params['strat_share_chartists'] = 0.0

In [34]:
trdrs_f = []
orbs_f = []
for seed in range(NRUNS): 
    traders, orderbook = init_objects_distr(params, seed)
    traders, orderbook = pb_distr_model(traders, orderbook, params, seed)
    trdrs_f.append(traders)
    orbs_f.append(orderbook)

Start of simulation  0
Start of simulation  1
Start of simulation  2
Start of simulation  3
Start of simulation  4
Start of simulation  5
Start of simulation  6
Start of simulation  7
Start of simulation  8
Start of simulation  9
Start of simulation  10
Start of simulation  11
Start of simulation  12
Start of simulation  13
Start of simulation  14
Start of simulation  15
Start of simulation  16
Start of simulation  17
Start of simulation  18
Start of simulation  19


In [38]:
# calc redistribution, profits ... abs 
profits_f = []

# determine the start and end wealth of the bubble
for seed, traders_f in enumerate(trdrs_f):
    money_start_f = np.array([x.var.money[0] for x in traders_f])
    stocks_start_f = np.array([x.var.stocks[0] for x in traders_f])
    wealth_start_f = money_start_f + (stocks_start_f * orbs_f[seed].tick_close_price[0])

    money_end_f = np.array([x.var.money[-1] for x in traders_f])
    stocks_end_f = np.array([x.var.stocks[-1] for x in traders_f])
    wealth_end_f = money_end_f + (stocks_end_f * orbs_f[seed].tick_close_price[-1])
    
    # profits
    profits_f.append((np.array(wealth_end_f) - np.array(wealth_start_f)) / np.array(wealth_start_f))

In [39]:
redistribution_f = []
for p in profits_f:
    redistribution_f.append(sum(abs(p)))

In [40]:
redistribution_f

[38.36935552788342,
 39.83874872951399,
 46.80463342904589,
 37.23051406010479,
 41.19369987388241,
 39.14739771520275,
 39.52996029578896,
 38.87585891833645,
 37.97093848787056,
 37.51314651156863,
 41.26318766790661,
 41.11907796980222,
 39.83356616678332,
 40.22153829359577,
 41.81677418650221,
 38.7388285043079,
 39.545002145197635,
 38.907014681483,
 42.11345687137009,
 42.102607065683564]

In [ ]:
# compare the 2

### Reason 2: Many small losses and a few big wins

In [47]:
# distribution of profits

pd.DataFrame({'profits_f': profits_f[0], 'profits': profits[0]}).skew()

profits_f    0.252738
profits      9.017681
dtype: float64